### 代理模式 

在某些应用中，我们想要在访问某个对象之前执行一个或多个重要的操作，例如，访问敏感
信息——在允许用户访问敏感信息之前，我们希望确保用户具备足够的权限。操作系统中也存在
类似的情况，用户必须具有管理员权限才能在系统中安装新程序。   
上面提到的重要操作不一定与安全问题相关。延迟初始化（请参考网页［t.cn/Ryf47bV］）
是另一个案例：我们想要把一个计算成本较高的对象的创建过程延迟到用户首次真正使用它时
才进行。      
这类操作通常使用代理设计模式（Proxy design pattern）来实现。该模式因使用代理（又名
替代，surrogate）对象在访问实际对象之前执行重要操作而得其名。以下是四种不同的知名代理
类型（请参考［GOF95，第234页］和网页［t.cn/RqrYEn9］）。    
 远程代理：实际存在于不同地址空间（例如，某个网络服务器）的对象在本地的代理者。  
 虚拟代理：用于懒初始化，将一个大计算量对象的创建延迟到真正需要的时候进行。 
 保护/防护代理：控制对敏感对象的访问。   
 智能（引用）代理：在对象被访问时执行额外的动作。此类代理的例子包括引用计数和
线程安全检查。    
我发现虚拟代理非常有用，所以现在通过一个例子来看看可以如何实现它。


使用Python来创建虚拟代理存在很多方式，但我始终喜欢地道的/符合Python风格的实现。这
里展示的代码源自网站stackoverflow.com用户Cyclone的一个超赞回答（请参考网页
［t.cn/RqrYudC］） 。          
为避免混淆，我先说明一下，在本节中，术语特性（property）、变量（variable）、
属性（attribute）可相互替代使用。我们先创建一个LazyProperty类，用作一个修饰器。当它
修饰某个特性时，LazyProperty惰性地（首次使用时）加载特性，而不是立即进行。__init__
方法创建两个变量，用作初始化待修饰特性的方法的别名。method变量是一个实际方法的别名，
method_name变量则是该方法名称的别名。为更好理解如何使用这两个别名，可以将其值输出
到标准输出

In [1]:
class LazyProperty:

    def __init__(self, method):
        self.method = method
        self.method_name = method.__name__
        # print('function overriden: {}'.format(self.method))
        # print("function's name: {}".format(self.method_name))

    def __get__(self, obj, cls):
        if not obj:
            return None
        value = self.method(obj)
        # print('value {}'.format(value))
        setattr(obj, self.method_name, value)
        return value


class Test:

    def __init__(self):
        self.x = 'foo'
        self.y = 'bar'
        self._resource = None

    @LazyProperty
    def resource(self):
        print('initializing self._resource which is: {}'.format(self._resource))
        self._resource = tuple(range(5))    # cost large
        return self._resource


def main():
    t = Test()
    print(t.x)
    print(t.y)

    # resource()方法当作一个变量（可以使用t.resource替代t.resource()
    print(t.resource)
    print(t.resource)

In [2]:
main()

foo
bar
initializing self._resource which is: None
(0, 1, 2, 3, 4)
(0, 1, 2, 3, 4)


LazyProperty类实际上是一个描述符（请参考网页［t.cn/RqrYBND］）。描述符（descriptor）
是Python中重写类属性访问方法（__get__()、__set__()和__delete__()）的默认行为要使
用的一种推荐机制。LazyProperty类仅重写了__set__()，因为这是其需要重写的唯一访问方
法。换句话说，我们无需重写所有访问方法。__get__()方法所访问的特性值，正是下层方法想
要赋的值，并使用setattr()来手动赋值。__get__()实际做的事情非常简单，就是使用值来
替代方法！这意味着不仅特性是惰性加载的，而且仅可以设置一次。我们马上就能看到这意味着
什么。同样，取消注释以上代码的的注释行，以得到一些额外信息

In [13]:
class LazyProperty:

    def __init__(self, method):
        self.method = method
        self.method_name = method.__name__
        print('function overriden: {}'.format(self.method))
        print("function's name: {}".format(self.method_name))

    def __get__(self, obj, cls):
        if not obj:
            return None
        #print(obj)
        value = self.method(obj)
        print('value {}'.format(value))
        setattr(obj, self.method_name, value)
        return value


class Test:

    def __init__(self):
        self.x = 'foo'
        self.y = 'bar'
        self._resource = None

    @LazyProperty
    def resource(self):
        print('initializing self._resource which is: {}'.format(self._resource))
        self._resource = tuple(range(5))    # 假设这一行的计算成本比较大
        return self._resource


def main():
    t = Test()
    #print(t)
    print(t.x)
    print(t.y)

    # resource()方法当作一个变量（可以使用t.resource替代t.resource()
    print(t.resource)
    print(t.resource)

function overriden: <function Test.resource at 0x0000000004C95BF8>
function's name: resource


In [14]:
main()

foo
bar
initializing self._resource which is: None
value (0, 1, 2, 3, 4)
(0, 1, 2, 3, 4)
(0, 1, 2, 3, 4)


从这个例子（文件lazy.py）的执行输出中，可以看出以下几点。     
 _resource变量实际不是在t实例创建时初始化的，而是在我们首次使用t.resource时。   ·  
 第二次使用t.resource之时，并没有再次初始化变量。这就是为什么初始化字符串
initializing self._resource which is:仅出现一次的原因。 

### 现实生活的例子 
芯片（又名芯片密码）卡（请参考网页［t.cn/RqrYdYx］ ）是现实生活中使用防护代理的一
个好例子。借记/信用卡包含一个芯片，ATM机或读卡器需要先读取芯片；在芯片通过验证后，
需要一个密码（PIN）才能完成交易。这意味着只有在物理地提供芯片卡并且知道密码时才能进
行交易。 
使用银行支票替代现金进行购买和交易是远程代理的一个例子。支票准许了对一个银行账户
的访问。

### 实现 
为演示代理模式，我们将实现一个简单的保护代理来查看和添加用户。该服务提供以下两个
选项。   
 查看用户列表：这一操作不要求特殊权限。    
 添加新用户：这一操作要求客户端提供一个特殊的密码。    
SensitiveInfo类包含我们希望保护的信息。users变量是已有用户的列表。read()方法
输出用户列表。add()方法将一个新用户添加到列表中  

In [1]:
class SensitiveInfo:

    def __init__(self):
        self.users = ['nick', 'tom', 'ben', 'mike']

    def read(self):
        print('There are {} users: {}'.format(len(self.users), ' '.join(self.users)))

    def add(self, user):
        self.users.append(user)
        print('Added user {}'.format(user))


class Info:

    # SensitiveInfo 保护代理

    def __init__(self):
        self.protected = SensitiveInfo()
        self.secret = '0xdeadbeef'

    def read(self):
        self.protected.read()
    #验证密码
    def add(self, user):
        sec = input('what is the secret? ')
        self.protected.add(user) if sec == self.secret else print("That's wrong!")


def main():
    info = Info()
    while True:
        print('1. read list |==| 2. add user |==| 3. quit')
        key = input('choose option: ')
        if key == '1':
            info.read()
        elif key == '2':
            name = input('choose username: ')
            info.add(name)
        elif key == '3':
            exit()
        else:
            print('unknown option: {}'.format(key))


In [ ]:
main()

1. read list |==| 2. add user |==| 3. quit
choose option: 1
There are 4 users: nick tom ben mike
1. read list |==| 2. add user |==| 3. quit


Info类是SensitiveInfo的一个保护代理。 secret变量值是客户端代码在添加新用户时被
要求告知/提供的密码。注意，这只是一个例子。现实中，永远不要执行以下操作。    
 在源代码中存储密码   
 以明文形式存储密码    
 使用一种弱（例如，MD5）或自定义加密形式   

你已经发现这个代理示例中可以改进的缺陷或缺失特性了吗？我有如下一些建议。      
 该示例有一个非常大的安全缺陷。没有什么能阻止客户端代码通过直接创建一个
SensitveInfo实例来绕过应用的安全设置。优化示例来阻止这种情况。一种方式是使用
abc模块来禁止直接实例化SensitiveInfo。在这种情况下，会要求进行其他哪些代码
变更呢？    
 一个基本的安全原则是，我们绝不应该存储明文密码。只要我们知道使用哪个库，安全
地存储密码并不是一件难事（请参考网页［t.cn/zQf0g3c］） 。如果你对安全感兴趣，阅读
这篇文章，并尝试实现一种外部存储密码的安全方式（例如，在一个文件或数据库中）。     
 应用仅支持添加新用户，那么如何删除一个已有用户呢？添加一个remove()方法。   
remove()应该是一个特权操作吗？   

### 应用案例   
因为存在至少四种常见的代理类型，所以代理设计模式有很多应用案例，如下所示。 
 在使用私有网络或云搭建一个分布式系统时。在分布式系统中，一些对象存在于本地内
存中，一些对象存在于远程计算机的内存中。如果我们不想本地代码关心两者之间的区
别，那么可以创建一个远程代理来隐藏/封装，使得应用的分布式性质透明化。    
 因过早创建计算成本较高的对象导致应用遭受性能问题之时。使用虚拟代理引入懒初始
化，仅在真正需要对象之时才创建，能够明显提高性能。    
 用于检查一个用户是否有足够权限来访问某个信息片段。如果应用要处理敏感信息（例
如，医疗数据），我们会希望确保用户在被准许之后才能访问/修改数据。一个保护/防护
代理可以处理所有安全相关的行为。    
 应用（或库、工具集、框架等）使用多线程，而我们希望把线程安全的重任从客户端代码转移到应用。这种情况下，可以创建一个智能代理，对客户端隐藏线程安全的复杂性。     
 对象关系映射（Object-Relational Mapping， ORM） API也是一个如何使用远程代理的例子。   
包括Django在内的许多流行Web框架使用一个ORM来提供类OOP的关系型数据库访问。
ORM是关系型数据库的代理，数据库可以部署在任意地方，本地或远程服务器都可以。